In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from get_data_yf import get_symbols, get_symbols_sample
import os

#TODO Change filepaths using os.pathjoint to avoid os dependency if you find time

RECOMMENDATION_WEIGHTS = [2, 1, 0, -1, -2] #weights for strongBuy, buy, hold, sell,	strongSell advisors respectively
TIMEZONE_WEIGHTS = [4, 3, 2, 1] # weights for 0, -1, -2, -3rd months respectively
ADVICE_COUNT = 50 #how many of best scored symbols we want to get data
folder = 'data/recommendations'
recommendation_data_folder = 'data/recommendation_data'

score_filename = 'scores.xlsx'
recommendations_filename = 'recommendations.xlsx'

In [ ]:
def create_weigth_kernel(vertical_weights, horizontal_weights):
    kernel_values = np.outer(vertical_weights, horizontal_weights)
    kernel = pd.DataFrame(kernel_values)
    kernel.rename(columns={0:'strongBuy', 1:'buy', 2:'hold', 
                              3:'sell', 4:'strongSell'}, inplace = True)
    
    return kernel

In [ ]:
def recommendation_score(recommendations, weight_kernel):
    #returns score and how many anaylst we have as advisor in total
    sum = recommendations.to_numpy().sum()
    if sum == 0:
        return None, None
    weighted_sum = (weight_kernel * recommendations).to_numpy().sum()
    return weighted_sum / sum, sum 

In [ ]:
def evaluate_symbols(symbols, timezone_weights, recommendation_weights):
    #Will return a df that stores scores and advisor count for a symbol
    scores = {}
    weight_kernel = create_weigth_kernel(timezone_weights, recommendation_weights)
    
    for symbol in symbols:
        recommendations = yf.Ticker(symbol).recommendations
        if  recommendations.empty:
            print(f"Couldn't fetch data for {symbol}.")
            continue
        
        recommendations = recommendations.drop('period', axis='columns')
        score, advisor = recommendation_score(recommendations, weight_kernel)
        if score:  
            scores[symbol] = [score, advisor]
        else: 
            print(f"No advisors found for {symbol}.") 
            
    scores_df = pd.DataFrame.from_dict(scores, orient='index', columns=['Scores', 'Advisors'])
    scores_df = scores_df.sort_values(by='Scores', ascending=False)
    
    return scores_df 

In [ ]:
def evaluate_files(recommendation_data_folder, timezone_weights, recommendation_weights):
    scores = {}
    weight_kernel = create_weigth_kernel(timezone_weights, recommendation_weights)
    
    files = os.listdir(recommendation_data_folder)
    for filename in files:
        if filename.endswith('.csv'):
            recommendations = pd.read_csv(f'{recommendation_data_folder}/{filename}')
            recommendations = recommendations.drop('period', axis='columns')
            
            score, advisor = recommendation_score(recommendations, weight_kernel)
            symbol = os.path.splitext(filename)[0]
            
            if score:  
                scores[symbol] = [score, advisor]
            else: 
                print(f"No advisors found for {symbol}.")
    
    scores_df = pd.DataFrame.from_dict(scores, orient='index', columns=['Scores', 'Advisors'])
    scores_df = scores_df.sort_values(by='Scores', ascending=False)
    
    return scores_df 

scores_df = evaluate_files(recommendation_data_folder, TIMEZONE_WEIGHTS, RECOMMENDATION_WEIGHTS)

In [ ]:
def write_recommendations(scores, folder, score_filename, recommendations_filename, advice_count=10):
    with pd.ExcelWriter(f'{folder}/{score_filename}', engine='xlsxwriter') as writer:
        scores.to_excel(writer)
        print("Score file saved.")
        
    wanted_symbols = scores.head(advice_count).index.tolist()
    with pd.ExcelWriter(f'{folder}/{recommendations_filename}', engine='xlsxwriter') as writer:
        starting_row = 0
        for symbol in wanted_symbols:
            recommendation = yf.Ticker(symbol).recommendations
            if not recommendation.empty:
                recommendation.to_excel(writer, startrow=starting_row)
                writer.sheets['Sheet1'].write(starting_row, 0, symbol)
                starting_row += len(recommendation) + 3
            else: 
                print(f"Couldn't write the recommendations for {symbol}.")
        print("Recommendation file saved.")

In [ ]:
def main_wo_data():
    #This is the main function to get files without recommendation dfs being 
    #present at the moment, it calls API for every symbol, so if we have too big
    #symbol list, it may trigger API limit. Use main_w_data if you have 
    #recommendation files in your hand
    symbols = get_symbols_sample() #Change this with get_symbols() when you want real -big- set
    scores = evaluate_symbols(symbols, TIMEZONE_WEIGHTS, RECOMMENDATION_WEIGHTS)
    write_recommendations(scores, folder, score_filename, recommendations_filename, ADVICE_COUNT)
    

In [ ]:
def main_w_data():
    scores = evaluate_files(recommendation_data_folder, TIMEZONE_WEIGHTS, RECOMMENDATION_WEIGHTS)
    write_recommendations(scores, folder, score_filename, recommendations_filename, ADVICE_COUNT)

In [ ]:
if __name__ == '__main__':
    pass

In [8]:
def get_symbols():
    #TODO get all available symbols from yahoo/ listing_status.csv
    #TODO save symbols to reduce api usage
    #TODO get only listed symbols
    
    df = pd.read_csv('data/listing_status.csv')
    symbols = df[['symbol']].items().to_list()
    print(symbols)
    
    #TODO GET ALL
get_symbols()

AttributeError: 'generator' object has no attribute 'to_list'